# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm

import math, collections
from scipy.stats import linregress

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope_s(y_axis):
    global window_s_ma
    x_axis = []
    for i in range(window_s_ma):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

def get_slope_l(y_axis):
    global window_l_ma
    x_axis = []
    for i in range(window_l_ma):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)


def get_month(row):
    month_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').date().month
    return(month_val)

def get_day(row):
    day_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').date().day
    return(day_val)   

def get_hour(row):
    hour_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').time().hour
    return(hour_val)   

def get_min(row):
    min_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').time().minute
    return(min_val)   


def get_dow(row):
    dow = dt.datetime.strptime(df['TS'][0], '%Y-%m-%d %H:%M').weekday()
    return(dow)

## Parameters

In [3]:
year = 2019

source_file_path = f'data\M1_{year}.csv'

min_bar = 10

pip_diff = 0.0003
diff_col = 'Close'

rsi_window = 10
rs_max = 1e6

window_s_ma = 10
window_l_ma = 50

## Read data

In [4]:
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

raw_df = pd.read_csv(source_file_path)

#raw_df['Close'] = (raw_df['High'] - raw_df['Low'])/2
#raw_df['tick'] = raw_df['Close']
#df = raw_df[['TS', 'Open', 'High', 'Low', 'Close']]

df = raw_df[['Open', 'High', 'Low', 'Close']]
print(f'{len(df)} records in df')
df.head()

372530 records in df


,Open,High,Low,Close
0,1.14598,1.14599,1.14598,1.14598
1,1.14598,1.14607,1.14598,1.14607
2,1.14607,1.14607,1.14606,1.14606
3,1.14606,1.14621,1.14606,1.14621
4,1.14619,1.14666,1.14604,1.14665


## Data manipulation

In [5]:
%%time
df = df.iloc[::min_bar]
print(f'{len(df)} records under {min_bar} min_bar')

#df['month'] = df.apply(get_month, axis=1)
##df['day'] = df.apply(get_day, axis=1)
#df['dow'] = df.apply(get_dow, axis=1)
#df['hour'] = df.apply(get_hour, axis=1)
#df['min'] = df.apply(get_min, axis=1)

df['diff'] = df['Close'].diff()

df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])

df['avg_gain'] = df['gain'].rolling(window=rsi_window).mean()
df['avg_loss'] = df['loss'].rolling(window=rsi_window).mean()

df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= rs_max, rs_max) 

df['rsi'] = 100 - (100 / (df['rs'] + 1))

df['ssma'] = df['Close'].rolling(window=window_s_ma).mean()
df['lsma'] = df['Close'].rolling(window=window_l_ma).mean()
df['sma_diff'] = df['ssma'] - df['lsma']

df['sema'] = df['Close'].ewm(span=window_s_ma).mean()
df['lema'] = df['Close'].ewm(span=window_l_ma).mean()

df['ema_diff'] = df['sema'] - df['lema']

#df['slope_s'] = df['Close'].rolling(window=window_s_ma).apply(get_slope_s)
#df['slope_l'] = df['Close'].rolling(window=window_l_ma).apply(get_slope_l)

#df['sma_slope'] = df['ssma'].rolling(window=rsi_window).apply(get_slope)

df['target'] = 'same'

df['target'].loc[df[diff_col].shift(-1) - df[diff_col] > pip_diff] = 'increase'
df['target'].loc[df[diff_col].shift(-1) - df[diff_col] < -pip_diff] = 'decrease'

df = df.dropna()
df = df.reset_index(drop=True)

37253 records under 10 min_bar
Wall time: 112 ms


## Write data to csv

In [6]:
#del df['TS']
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

## Print Report

In [7]:
target_col = 'target'

print(f'Record count : {len(df)}')
print('--------------------------')
print(df[target_col].value_counts())
print('--------------------------')
print(df[target_col].value_counts(normalize=True))

df.head(10)

Record count : 37204
--------------------------
same        29852
decrease     3722
increase     3630
Name: target, dtype: int64
--------------------------
same        0.802387
decrease    0.100043
increase    0.097570
Name: target, dtype: float64


,Open,High,Low,Close,diff,gain,loss,avg_gain,avg_loss,rs,rsi,ssma,lsma,sma_diff,sema,lema,ema_diff,target
0,1.14916,1.14943,1.14909,1.14919,0.00070,0.00070,0.00000,0.000499,0.000007,71.285714,98.616601,1.146126,1.145550,0.000576,1.146690,1.145560,0.001131,same
1,1.14941,1.14942,1.14916,1.14917,-0.00002,0.00000,0.00002,0.000474,0.000009,52.666667,98.136646,1.146591,1.145613,0.000978,1.147141,1.145722,0.001419,same
2,1.14917,1.14917,1.14909,1.14912,-0.00005,0.00000,0.00005,0.000474,0.000007,67.714286,98.544699,1.147058,1.145674,0.001384,1.147501,1.145875,0.001626,same
3,1.14885,1.14897,1.14882,1.14885,-0.00027,0.00000,0.00027,0.000450,0.000034,13.235294,92.975207,1.147474,1.145730,0.001744,1.147746,1.146007,0.001739,same
4,1.14890,1.14899,1.14870,1.14870,-0.00015,0.00000,0.00015,0.000445,0.000049,9.081633,90.080972,1.147870,1.145783,0.002087,1.147920,1.146127,0.001793,decrease
5,1.14840,1.14843,1.14827,1.14829,-0.00041,0.00000,0.00041,0.000390,0.000090,4.333333,81.250000,1.148170,1.145823,0.002347,1.147987,1.146222,0.001765,same
6,1.14869,1.14869,1.14850,1.14856,0.00027,0.00027,0.00000,0.000347,0.000090,3.855556,79.405034,1.148427,1.145874,0.002553,1.148091,1.146325,0.001767,same
7,1.14869,1.14894,1.14849,1.14853,-0.00003,0.00000,0.00003,0.000258,0.000093,2.774194,73.504274,1.148592,1.145913,0.002679,1.148171,1.146421,0.001750,decrease
8,1.14806,1.14815,1.14806,1.14809,-0.00044,0.00000,0.00044,0.000244,0.000137,1.781022,64.041995,1.148699,1.145943,0.002756,1.148156,1.146494,0.001663,same
9,1.14832,1.14832,1.14813,1.14814,0.00005,0.00005,0.00000,0.000102,0.000137,0.744526,42.677824,1.148664,1.145974,0.002690,1.148153,1.146565,0.001588,same
